In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [14]:
H = []
# T0 = []
for i in range(3):
    H.append(xr.open_dataset('ERA5/htrlwcs'+str(90-10*i))['avg_ttlwrcs'])
    # T0.append(T[i][0].data)

In [15]:
H[0].shape[0]

120

In [16]:
dat = xr.open_dataset('ERA5/t_2000-2018.nc')
tfs = pd.read_csv('model_to_p.csv')

In [17]:
# Load data

# Target pressure levels (standard pressure levels)
p_target = dat.level.data  # shape (n_levels,)

# Source pressure levels for model levels (from CSV)
pm = tfs['P'].values  # shape (model_levels,)

# H[0] — heating rate: shape (time, model_level)
# H0 = dat.t.data  # shape: (time, model_level)
for i in range(3):
        
    # Output container
    H_interp = np.empty((H[i].shape[0], len(p_target)))  # (time, new_levels)

    # Interpolate each time step
    for t in range(H[i].shape[0]):
        f = interp1d(pm, H[i][t, :], bounds_error=False, fill_value='extrapolate')
        H_interp[t, :] = f(p_target)

    # Wrap in DataArray
    H_interp_da = xr.DataArray(
        H_interp,
        dims=['time', 'level'],
        coords={'time': H[i]['valid_time'].data, 'level': p_target},
        name='avg_ttlwrcs'
    )
    H_interp_da.to_netcdf('ERA5/htrlwcs_inp'+str(90-10*i)+'.nc')
